* Quando estamos alterando a ordem dos dados de treino e teste, podemos quebrar em k pedaços de tal forma que , caso sejam:
    * 2 pedaços:: 1°: primeira metade treina e segunda testa && 2°: primeira metade testa e segunda treina
    * 3 pedaços:: 1°: primeiro pedaço treina e resto testa && segundo pedaço treina e o resto testa && terceiro pedaço treina e o resto testa
*  Ou seja, realizo os testes para CADA UMA DAS PARTIÇÕES 

In [1]:
from typing import Counter
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score # já vai fazar o fit, predict e devolver as notas

In [4]:
df = pd.read_csv('clientes.csv')
X_df = df[['recencia','frequencia', 'semanas_de_inscricao']]
Y_df = df['situacao']

In [6]:
Xdummies_df = pd.get_dummies(X_df)
Ydummies_df = Y_df

In [9]:
X = Xdummies_df.values
Y = Ydummies_df.values 

In [11]:
porcentagem_de_treino_e_teste = 0.9

tamanho_de_treino_e_teste = int(porcentagem_de_treino_e_teste * len(Y))

treino_dados = X[:tamanho_de_treino_e_teste]
treino_marcacoes = Y[:tamanho_de_treino_e_teste]

validacao_dados = X[tamanho_de_treino_e_teste:]
validacao_marcacoes = Y[tamanho_de_treino_e_teste:]

In [12]:
def fit_and_predict(nome, modelo, treino_dados, treino_marcacoes):
    k = 10
    scores = cross_val_score(modelo, treino_dados, treino_marcacoes, cv = k)
    taxa_de_acerto = np.mean(scores)
    msg = f'A taxa de acerto de {nome} foi: {taxa_de_acerto}'
    print(msg)
    return(taxa_de_acerto)

def teste_real(modelo, validacao_dados, validacao_marcacoes):
    resultado = modelo.predict(validacao_dados)

    acertos = resultado == validacao_marcacoes

    total_de_acertos = sum(acertos)
    total_de_elementos = len(validacao_marcacoes)

    taxa_de_acerto = 100.0 * total_de_acertos / total_de_elementos

    msg = "Taxa de acerto do vencedor entre os algoritmos no mundo real: {0}".format(taxa_de_acerto)
    print(msg)

In [13]:
resultados = {}

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
modeloOneVsRest = OneVsRestClassifier(LinearSVC(random_state = 0, max_iter=10000))
resultadoOneVsRest = fit_and_predict("OneVsRest", modeloOneVsRest, treino_dados, treino_marcacoes)
resultados[resultadoOneVsRest] = modeloOneVsRest

from sklearn.multiclass import OneVsOneClassifier
modeloOneVsOne = OneVsOneClassifier(LinearSVC(random_state = 0, max_iter=10000))
resultadoOneVsOne = fit_and_predict("OneVsOne", modeloOneVsOne, treino_dados, treino_marcacoes)
resultados[resultadoOneVsOne] = modeloOneVsOne

from sklearn.naive_bayes import MultinomialNB
modeloMultinomial = MultinomialNB()
resultadoMultinomial = fit_and_predict("MultinomialNB", modeloMultinomial, treino_dados, treino_marcacoes)
resultados[resultadoMultinomial] = modeloMultinomial

from sklearn.ensemble import AdaBoostClassifier
modeloAdaBoost = AdaBoostClassifier()
resultadoAdaBoost = fit_and_predict("AdaBoostClassifier", modeloAdaBoost, treino_dados, treino_marcacoes)
resultados[resultadoAdaBoost] = modeloAdaBoost

A taxa de acerto de OneVsRest foi: 0.9257142857142858
A taxa de acerto de OneVsOne foi: 1.0
A taxa de acerto de MultinomialNB foi: 0.8366666666666667
A taxa de acerto de AdaBoostClassifier foi: 0.7528571428571429


In [14]:
for r in resultados.items():
    print(r)


maximo = max(resultados)
vencedor = resultados[maximo]

print("Vencerdor: ")
print(vencedor)

vencedor.fit(treino_dados, treino_marcacoes)

teste_real(vencedor, validacao_dados, validacao_marcacoes)

acerto_base = max(Counter(validacao_marcacoes).values())
taxa_de_acerto_base = 100.0 * acerto_base / len(validacao_marcacoes)
print("Taxa de acerto base: %f" % taxa_de_acerto_base)

total_de_elementos = len(validacao_dados)
print("Total de teste: %d" % total_de_elementos)

(0.9257142857142858, OneVsRestClassifier(estimator=LinearSVC(max_iter=10000, random_state=0)))
(1.0, OneVsOneClassifier(estimator=LinearSVC(max_iter=10000, random_state=0)))
(0.8366666666666667, MultinomialNB())
(0.7528571428571429, AdaBoostClassifier())
Vencerdor: 
OneVsOneClassifier(estimator=LinearSVC(max_iter=10000, random_state=0))
Taxa de acerto do vencedor entre os algoritmos no mundo real: 100.0
Taxa de acerto base: 82.608696
Total de teste: 23
